# Materia prima, costos y ventas

Crearemos un dataframe que sea de utilidad para hacer un analisis enfocado hacia los productos. El dataframe madre contiene todos los registros y columnas obtenidas por cada articulo incluido en una orden de compra.

In [63]:
import pandas as pd
import numpy as np

In [68]:
df = pd.read_csv('operaciones.csv', ).reset_index(drop=True)

In [69]:
# Creamos un diccionario con las columnas que vamos a necesitar

material = ['foliointerno',
'Usuario.razonSocial', 
'Productos.idLocal',
'Productos.unidades',
'Productos.cantidad',
'Productos.importe',
'Especificaciones.tipo',
'Especificaciones.calibreECT',
'Especificaciones.caraExterior',
'Especificaciones.largo',
'Especificaciones.ancho',
'Especificaciones.alto',
'Especificaciones.tipoUnion',
'Especificaciones.suaje']

# Introducimos el diccionario como filtro para ordenar

material = df[material]
material['Productos.importe'].count()

237

In [62]:
material[material['Especificaciones.calibreECT'] == 23]

,foliointerno,Usuario.razonSocial,Productos.idLocal,Productos.unidades,Productos.cantidad,Productos.importe,Especificaciones.tipo,Especificaciones.calibreECT,Especificaciones.caraExterior,Especificaciones.largo,Especificaciones.ancho,Especificaciones.alto,Especificaciones.tipoUnion,Especificaciones.suaje,Especificaciones.longitudLamina,Especificaciones.anchoLamina,areaM2
24,1IN-00729,"PLASTONIUM, S.A DE C.V.",CAJA NORMAL TIPO 3P,millares,1.5,12105.0,"[5.5, 1.5]",23,kraft,37.0,33.5,16.9,CIP,0,1.465,1.023,1.498695


El corrugado se construye a partir de m2 de lamina producida de un mismo calibre por lo que crearemos un dataframe con solo las columnas de la especificación del material añadiendo una columna del area de la lamina por m2, segun la siguiente formula:

               L(lamina) = ((2*L) * (2*W)) + desarrollo
               W(lamina) = (L*W) + Desarrollo
                    
La variable de desarrollo se asigna dependiendo del tipo de corrugado de acuerdo a la siguiente tabla:

                    {'regular': 5.5,
                    'dosPiezas': 1.5,
                    'N/A': 0}

In [58]:
# Creamos el diccionario para sustituir la llave del tipo con el desarrollo de tolerancia.

desarrollo = {'regular': [5.5, 1.5],
                    'dosPiezas': [7, 2.7],
                    np.nan: [0, 0]}

# Sustituyendo y aplicando el diccionario en la columna

material["Especificaciones.tipo"] = material["Especificaciones.tipo"].map(desarrollo)

<ipython-input-58-fdd7339171be>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  material["Especificaciones.tipo"] = material["Especificaciones.tipo"].map(desarrollo)


In [59]:
# Crear funciones para calcular la longitud y ancho totales de la lamina

def largoLaminaM2(largo, ancho, desarrollo):
    longitudLamina = ((2*largo) + (2*ancho)) + desarrollo
    return longitudLamina / 100

def anchoLaminaM2(ancho, alto, desarrollo):
    anchoLamina = (ancho + alto) + desarrollo
    return anchoLamina / 100

Creamos las columnas aplicando las funciones

In [60]:
# Largo de lamina

material["Especificaciones.longitudLamina"] = largoLaminaM2(material["Especificaciones.largo"], material["Especificaciones.ancho"], material["Especificaciones.tipo"].str[0])

# Ancho de lamina

material["Especificaciones.anchoLamina"] = largoLaminaM2(material["Especificaciones.ancho"], material["Especificaciones.alto"], material["Especificaciones.tipo"].str[1])

# Área total en metros cuadrados

material["areaM2"] = material["Especificaciones.anchoLamina"] * material["Especificaciones.longitudLamina"]
#material["totalVenta"] = material["areaM2"] * material["precio.precio"]

<ipython-input-60-1224e033d077>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  material["Especificaciones.longitudLamina"] = largoLaminaM2(material["Especificaciones.largo"], material["Especificaciones.ancho"], material["Especificaciones.tipo"].str[0])
<ipython-input-60-1224e033d077>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  material["Especificaciones.anchoLamina"] = largoLaminaM2(material["Especificaciones.ancho"], material["Especificaciones.alto"], material["Especificaciones.tipo"].str[1])
<ipyth

In [53]:
material.groupby('Especificaciones.calibreECT').last()

,foliointerno,Usuario.razonSocial,Productos.idLocal,Productos.unidades,Productos.cantidad,Productos.importe,Especificaciones.tipo,Especificaciones.caraExterior,Especificaciones.largo,Especificaciones.ancho,Especificaciones.alto,Especificaciones.tipoUnion,Especificaciones.suaje,Especificaciones.longitudLamina,Especificaciones.anchoLamina,areaM2
Especificaciones.calibreECT,,,,,,,,,,,,,,,,
23,1IN-00729,"PLASTONIUM, S.A DE C.V.",CAJA NORMAL TIPO 3P,millares,1.5,12105.0,"[5.5, 1.5]",kraft,37.0,33.5,16.9,CIP,0,1.465,1.023,1.498695
26,3685-354,CREMERIA COVADONGA S.A. DE C.V.,panela premium separador,millares,5.0,13600.0,NaN,kraft,29.0,29.0,0.0,NaN,0,NaN,NaN,NaN
29,24151,LOQUAY. S.A.,CJ60,millares,3.0,16536.0,"[5.5, 1.5]",kraft,29.0,20.5,16.3,CIP,0,1.045,0.751,0.784795
32,0000001428,"Intermuebles Modulares, S.A. de C.V.",CAJA-NOC4D,millares,0.2,21000.0,"[5.5, 1.5]",kraft,74.8,48.0,97.0,CIP,0,2.511,2.915,7.319565


In [61]:
analisisECT = material.groupby('Especificaciones.calibreECT')[['Productos.cantidad','areaM2']].sum()
analisisECT

,Productos.cantidad,areaM2
Especificaciones.calibreECT,,
23,1.5,1.498695
26,270.0,36.331200
29,38.5,8.430165
32,331.0,489.666709
